Import related libraries

In [2]:
import os.path

import openai
from src.utilities import *
import random

You must add your own OpenAI API key to the `openai-api-key.txt` file.

In [6]:
api_key = get_file_contents("openai-api-key.txt")

Initialize the OpenAI API Client with the API key

In [7]:
openai.api_key = api_key

Set your training mode for the fine-tuning process, all files related to this mode will have the name of the mode in their names.

In [33]:
train_mode = "100-input-10-epoch-0.5-temp-50-output"

Get hyperparameters for the fine-tuning process

In [9]:
hypers = read_hypers()

Convert training data to JSONL format

In [34]:
training_data_folder = f"data/training_data/{hypers[train_mode]["training_set"]}"
training_data_path = f"{training_data_folder}/training_data.jsonl"
create_jsonl_from_folder(training_data_folder, training_data_path)

Upload the training file to the API for fine-tuning purposes

In [35]:
with open(training_data_path, "r") as file:
    response = openai.File.create(file=file, purpose="fine-tune")

file_id = response["id"]
write_file_contents(f"data/client_file_ids/{train_mode}.txt", file_id)

Set the base model to fine-tune. gpt-3.5-turbo-0125 is the currently recommended model.

In [12]:
base_model = "gpt-3.5-turbo-0125"

Create a fine-tuning job for the model with the training file

In [36]:
training_file_id = get_file_contents(f"data/client_file_ids/{train_mode}.txt")

response = openai.FineTuningJob.create(
    model=base_model,
    training_file=training_file_id,
    suffix=train_mode.replace("-input", "").replace("-epoch", "").replace("-temp", "").replace("-output", ""),
    hyperparameters={"n_epochs": hypers[train_mode]["n_epochs"]}
)

fine_tuning_job_id = response["id"]
write_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt", fine_tuning_job_id)

Generate levels using the fine-tuned model

In [24]:
fine_tuning_job_id = get_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt")
fine_tuning_job = openai.FineTuningJob.retrieve(id=fine_tuning_job_id)

fine_tuning_status = fine_tuning_job.status
if fine_tuning_status == "succeeded":
    fine_tuned_model = fine_tuning_job.fine_tuned_model
    
    for i in range(hypers[train_mode]["n_generations"]):
        m = random.randint(7, 35)
        n = random.randint(8, 35)
        response = openai.ChatCompletion.create(
            model=fine_tuned_model,
            messages=get_messages_for_chat_completion(m, n),
            temperature=hypers[train_mode]["temperature"],
        )
        generated_level = response.choices[0].message["content"]
        
        generated_level_folder = f"data/generated_levels/{train_mode}"
        if not os.path.exists(generated_level_folder):
            os.makedirs(generated_level_folder)
        write_file_contents(f"{generated_level_folder}/level_{i}_{m}x{n}.txt", generated_level)
        print(f"Generated level {i} with dimensions {m}x{n} for {train_mode}")
else:
    print(f"Fine-tuning job {fine_tuning_job_id} has not succeeded yet. Current status: {fine_tuning_status}")

Generated level 0 with dimensions 8x30 for 10-input-5-epoch-0.5-temp-50-output
Generated level 1 with dimensions 33x32 for 10-input-5-epoch-0.5-temp-50-output
Generated level 2 with dimensions 21x21 for 10-input-5-epoch-0.5-temp-50-output
Generated level 3 with dimensions 17x12 for 10-input-5-epoch-0.5-temp-50-output
Generated level 4 with dimensions 24x29 for 10-input-5-epoch-0.5-temp-50-output
Generated level 5 with dimensions 23x26 for 10-input-5-epoch-0.5-temp-50-output
Generated level 6 with dimensions 30x11 for 10-input-5-epoch-0.5-temp-50-output
Generated level 7 with dimensions 14x23 for 10-input-5-epoch-0.5-temp-50-output
Generated level 8 with dimensions 13x21 for 10-input-5-epoch-0.5-temp-50-output
Generated level 9 with dimensions 35x11 for 10-input-5-epoch-0.5-temp-50-output
Generated level 10 with dimensions 33x24 for 10-input-5-epoch-0.5-temp-50-output
Generated level 11 with dimensions 16x22 for 10-input-5-epoch-0.5-temp-50-output
Generated level 12 with dimensions 23x1

In [3]:
print(check_level_validity(read_level_from_txt("data/generated_levels/200-input-3-epoch-0.5-temp-50-output/level_0_8x12.txt")))
print(check_level_validity(read_level_from_txt("data/training_data/training-set-200-examples/Sasquatch_level_35.txt")))

(False, 'The number of crates and storage locations should be equal.')
None
